In [1]:
# MIT image
# Made by : HERO Kwon
# Date : 190108

import os
import numpy as np
import pandas as pd
import pickle
import gzip
import matplotlib.pyplot as plt
import math
import cmath

# data path
#path_meta = '/home/herokwon/mount_data/Data/Wi-Fi_meta/'
#path_csi = '/home/herokwon/mount_data/Data/Wi-Fi_processed/'
#path_csi_np = '/home/herokwon/mount_data/Data/Wi-Fi_processed_npy/'

# data path_mi
path_csi = 'J:\\Data\\Wi-Fi_processed\\'
path_csi_np = 'J:\\Data\\Wi-Fi_processed_npy\\'
path_meta = 'J:\\Data\\Wi-Fi_meta\\'
path_sc = 'J:\\Data\\Wi-Fi_info\\'


In [2]:
# data info
df_info = pd.read_csv(path_meta+'data_subc_sig_v1.csv')
person_uid = np.unique(df_info['id_person'])
dict_id = dict(zip(person_uid,np.arange(len(person_uid))))
csi_time = 15000 #int(np.max(df_info['len']))
# parameters
max_value = np.max(df_info['max'].values)
#no_classes = len(np.unique(df_info['id_person']))
no_classes = len(dict_id)
csi_subc = 30
input_shape = (csi_time, csi_subc, 6)

# freq BW list
bw_list = pd.read_csv(path_meta+'wifi_f_bw_list.csv')

# avg Array
with open(path_meta + 'dict_avgcsi.pickle','rb') as f:
    dict_avg = pickle.load(f)


In [3]:

# make data generator
def gen_csi(df_info,dict_avg,id_num,len_num):
    for file in np.unique(df_info.id.values):
        
        # Label
        id_key = df_info[df_info.id==file]['id_person'].values[0].astype('int')
        data1_y = dict_id[id_key]
        
        # read sample data
        # load and uncompress.
        with gzip.open(path_csi+file+'.pickle.gz','rb') as f:
            data1 = pickle.load(f)
            
        # subtract avg
        read_avg = dict_avg[id_key][0]
        nonzero_avg = read_avg[np.nonzero(read_avg)].reshape([-1,30,2,3])
        arr_avg = np.mean(nonzero_avg,axis=0)
        data1_diff = data1 - arr_avg #np.diff(data1,axis=0)
        # zero pad
        pad_len = len_num - data1_diff.shape[0]
        data1_pad = np.pad(data1_diff,((0,pad_len),(0,0),(0,0),(0,0)),'constant',constant_values=0)

        # subcarrier info
        data1_sc_df = pd.read_csv(path_sc + file + '_df_sc.csv')
        data1_time = np.diff(data1_sc_df['timestamp_low'])
        data1_time_pad = np.pad(data1_time,((1,pad_len)),'constant',constant_values=0)
        
        
        yield(data1_pad ,data1_y,id_key,data1_time_pad)


gen = gen_csi(df_info,dict_avg,no_classes,csi_time)
target_sig,target_lab,target_id,target_dt = next(gen)


In [4]:
# 3D scan
m,n = 2,3
c =  299792458 # speed of light 
#r = (160 + 160 + 164) * 0.01 # meter
r = 1.64 #meter
d = 45 * 0.01 # meter
ch = 8#3
max_subc = 3

th_range,si_range = (30,30)
sig_mat = np.zeros([max_subc,csi_time,2*th_range,2*si_range])


In [ ]:
# 3D Reconstruction func
from numba import vectorize
@vectorize(['complex64(complex64,float32,float32,int32,int32,\
float32,float32,float32,float32,int64,int32)'], target='cuda')
def Recon3d(sig,theta,sigma,m,n,lam,d,k,r,dt,c):
    #above_eq1 = 1j * (2*math.pi) * k * r * t / c
    above_eq1 = 1j * (2*math.pi) * c * dt / lam
    above_eq2 = 1j * (2*math.pi/lam) * math.sin(theta) * ((n+1)*d*math.cos(sigma) + (m+1)*d*math.sin(sigma))
    eq_res = sig* cmath.exp(above_eq1) * cmath.exp(above_eq2)
    return eq_res #math.atan2(eq_res.imag,eq_res.real)


In [ ]:
from numba import jit
@jit
def Calc3d(target_sig,max_subc,th_range,si_range,bw_list,ch,target_dt,\
          d,r,c):
    # Calc 3D
    for subc in range(max_subc):
        sig1 = target_sig[:,subc,:,:]
        for idx_th,i in enumerate(range(-th_range,th_range)):
            for idx_si,j in enumerate(range(-si_range,si_range)):
                lam =  c*0.000001 / bw_list[str(subc)][ch] #wavelength = 300 / frequency in MHz
                #t = np.arange(1,csi_time+1,1,dtype=np.int32)
                dt = target_dt
                k = 0.0
                theta = i * (np.radians(90)/2) / th_range
                sigma = j * (np.radians(90)/2) / si_range
                sum_eq = np.zeros(csi_time,dtype=np.complex_)
                for m in [0,1]: 
                    for n in [0,1,2]: 
                        #above_eq1 = Calc_Above1(k,r,t,c)
                        sig1 = np.ascontiguousarray(target_sig[:,subc,m,n], dtype=np.complex64)
                        sum_eq += Recon3d(sig1,theta,sigma,m,n,lam,d,k,r,dt,c)
                sig_mat[subc,:,idx_th,idx_si] =  np.abs(sum_eq)
    return(sig_mat)


In [ ]:
import time
t1 = time.time()
sig_mat = Calc3d(target_sig,max_subc,th_range,si_range,bw_list,ch,target_dt,\
          d,r,c)
print(time.time()-t1)

C:\Users\herok\Anaconda3\envs\mit_image\lib\site-packages\numba\cuda\compiler.py:233: UserWarning: Could not autotune, using default tpb of 128
  warnings.warn('Could not autotune, using default tpb of 128')
